In [8]:
import deepchem as dc
import pandas as pd
import pickle
import numpy as np
import warnings

from utils import *
warnings.filterwarnings('ignore')
file = open('tests/tip3p/model_10/results.pickle','rb')
results = pickle.load(file)
stats = results['stats'][0]
stats

{'phy_rmsd': {'train': 1.5560596697201357, 'test': 1.4352081556345755},
 'ml_rmsd': {'train': 0.6939260760419648, 'test': 1.0909239381377032},
 'phy_md': {'train': -0.33443594306049823, 'test': -0.19277499999999997},
 'ml_md': {'train': 0.12944947844955845, 'test': 0.37069442521631724},
 'phy_out_rmsd': {'train': 4.825227548681173, 'test': 3.7860636880538605},
 'ml_out_rmsd': {'train': 2.219233030948551, 'test': 3.490601617748609},
 'params': {'epochs': 500,
  'dropout': 0.4,
  'batch_normalize': False,
  'batch_size': 100,
  'feat': 'tip3p',
  'kfold': 20,
  'dense_layer_size': 27,
  'graph_conv_layers': [32, 32],
  'model_dir': 'model_10'}}

In [ ]:
# params = [{'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'tip3p', 
#            'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
#           {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'cha', 
#           'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
#          {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'gbnsr6', 
#           'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
#          {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'asc', 
#           'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
#          {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'igb5', 
#           'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},   
#          {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'null', 
#           'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]}
#          ]

In [13]:
error = 1

feats = ['tip3p','cha','gbnsr6','asc','igb5','null']
for feat in feats:
    print(feat)
    phys_rmsd_train = []
    phys_rmsd_test = []
    ml_rmsd_train = []
    ml_rmsd_test = []
    for i in range(100):
        file = open('tests/'+feat+'/model_'+str(i)+'/results.pickle','rb')
        
        results = pickle.load(file)
        stats = results['stats'][0]
        phys_rmsd_test.append(stats['phy_rmsd']['test'])
        phys_rmsd_train.append(stats['phy_rmsd']['train'])
        ml_rmsd_test.append(stats['ml_rmsd']['test'])
        ml_rmsd_train.append(stats['ml_rmsd']['train'])
    print('physics model: test',np.round(np.mean(phys_rmsd_test),3),'±',np.round(error*np.std(phys_rmsd_test),3),
              'train',np.round(np.mean(phys_rmsd_train),3),'±',np.round(error*np.std(phys_rmsd_train),3))
    print('physics + ml: test',np.round(np.mean(ml_rmsd_test),3),'±',np.round(error*np.std(ml_rmsd_test),3),
              'train',np.round(np.mean(ml_rmsd_train),3),'±',np.round(error*np.std(ml_rmsd_train),3))

tip3p
physics model: test 1.435 ± 0.0 train 1.556 ± 0.0
physics + ml: test 1.038 ± 0.071 train 0.707 ± 0.048
cha
physics model: test 1.315 ± 0.0 train 1.77 ± 0.0
physics + ml: test 0.991 ± 0.058 train 0.775 ± 0.04
gbnsr6
physics model: test 1.855 ± 0.0 train 1.64 ± 0.0
physics + ml: test 1.459 ± 0.072 train 0.756 ± 0.046
asc
physics model: test 2.652 ± 0.0 train 2.489 ± 0.0
physics + ml: test 1.573 ± 0.096 train 1.038 ± 0.061
igb5
physics model: test 3.112 ± 0.0 train 2.805 ± 0.0
physics + ml: test 1.658 ± 0.162 train 1.039 ± 0.076
null
physics model: test 5.45 ± 0.0 train 5.402 ± 0.0
physics + ml: test 1.633 ± 0.118 train 1.12 ± 0.095
